In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
from tqdm import tqdm

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [ ]:
serval_data_folder = "../data"

In [ ]:
# Set directories of WAV files
youtube_wav_samples_directory = serval_data_folder + "/wav_samples_youtube"

youtube_class_labels_filepath = serval_data_folder + "/csv_files/class_labels_indices.csv"

youtube_wav_balanced_train_class_labels_filepath   = youtube_wav_samples_directory + "/balanced_train_segments.csv"
youtube_wav_balanced_eval_class_labels_filepath    = youtube_wav_samples_directory + "/eval_segments.csv"
youtube_wav_unbalanced_train_class_labels_filepath = youtube_wav_samples_directory + "/unbalanced_train_segments.csv"

youtube_wav_balanced_train_sample_directory   = youtube_wav_samples_directory + "/bal"
youtube_wav_balanced_eval_sample_directory    = youtube_wav_samples_directory + "/eval"
youtube_wav_unbalanced_train_sample_directory = youtube_wav_samples_directory + "/unbal"

In [ ]:
# Set output csv files
target_balanced_wav_samples_enumerated_filepath   = serval_data_folder + "/csv_files/wav_samples_youtube_balanced_all_enumerated_and_labeled.csv"
target_unbalanced_wav_samples_enumerated_filepath = serval_data_folder + "/csv_files/wav_samples_youtube_unbalanced_all_enumerated_and_labeled.csv"
target_eval_wav_samples_enumerated_filepath       = serval_data_folder + "/csv_files/wav_samples_youtube_eval_all_enumerated_and_labeled.csv"

In [ ]:
def collectAndLabelYoutubeSamples(df_youtube_class_labels, wav_class_labels_filepath, wav_sample_directory, source):
    df_wav_class_labels = pd.read_csv(wav_class_labels_filepath, sep=",")    
    df_output_wav_files_classified = pd.DataFrame(columns=df_youtube_class_labels.columns.values.tolist() + ['source', 'filename', 'filepath'])
    
    start_time = time.time()
    files = gfile.Glob(str(wav_sample_directory + "/" + "*.wav"))
    for file in tqdm(files):
        filename = ntpath.basename(file)[3:-4]
        df_labels = df_wav_class_labels.loc[df_wav_class_labels.filename.isin([filename])]
        
        if len(df_labels) != 1:
            print('[ERROR]: Found sample with ', len(df_labels) ,' class labels records (expected=1); filename=', filename, '; path=', file, sep='')
            continue
        
        classes = str(df_labels.youtube_mid_labels.iloc[0]).split(";")
        for class_ in classes:
            df_row = df_youtube_class_labels.loc[df_youtube_class_labels.mid == class_].copy(deep=True)
            
            if len(df_row) != 1:
                print('[ERROR]: Found sample with ', len(df_row) ,' youtube class labels records (expected=1); filename=', filename, '; youtube_mid_id=', class_, sep='')
                continue
            
            df_row['source'] = source
            df_row['filename'] = filename
            df_row['filepath'] = file
            df_output_wav_files_classified = df_output_wav_files_classified.append(df_row, ignore_index = True)         
    
    df_output_wav_files_classified = df_output_wav_files_classified.rename(columns={'index': 'label'})
    return(df_output_wav_files_classified)
    

In [ ]:
# Load generic youtube class labels
df_youtube_class_labels = pd.read_csv(youtube_class_labels_filepath, sep=",")
df_youtube_class_labels.head()

In [ ]:
#  collect and label balanced train samples
df_tmp = collectAndLabelYoutubeSamples(df_youtube_class_labels, 
                                       youtube_wav_balanced_train_class_labels_filepath,
                                       youtube_wav_balanced_train_sample_directory,
                                      'wav_samples_youtube/bal')
df_tmp.to_csv(target_balanced_wav_samples_enumerated_filepath, sep=';')

display(df_tmp.agg(['nunique']))
display(df_tmp.head())

In [ ]:
#  collect and label balanced eval samples
df_tmp = collectAndLabelYoutubeSamples(df_youtube_class_labels, 
                                       youtube_wav_balanced_eval_class_labels_filepath,
                                       youtube_wav_balanced_eval_sample_directory,
                                      'wav_samples_youtube/eval')
df_tmp.to_csv(target_eval_wav_samples_enumerated_filepath, sep=';')

display(df_tmp.agg(['nunique']))
display(df_tmp.head())

In [ ]:
#  collect and label unbalanced train samples
df_tmp = collectAndLabelYoutubeSamples(df_youtube_class_labels, 
                                       youtube_wav_unbalanced_train_class_labels_filepath,
                                       youtube_wav_unbalanced_train_sample_directory,
                                      'wav_samples_youtube/unbal')
df_tmp.to_csv(target_unbalanced_wav_samples_enumerated_filepath, sep=';')

display(df_tmp.agg(['nunique']))
display(df_tmp.head())